In [3]:
# build chat input prompt
def build_chat_input(tokenizer, message):
    # chat format:
    # single-turn: <s>Human: Hello!\n\nAssistant: </s>

    prompt = "Human: " + message + "\n\nAssistant:"

    input_tokens = tokenizer.encode(prompt, return_tensors="pt")
    return input_tokens

In [4]:
from datasets import load_dataset

dataset = load_dataset("narrativeqa")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [5]:
dataset['train'][0]['document']['text'][3:1000]

'The Project Gutenberg EBook of Percival Keene, by Frederick Marryat\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Percival Keene\n\nAuthor: Frederick Marryat\n\nRelease Date: May 22, 2007 [EBook #21572]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PERCIVAL KEENE ***\n\n\n\n\nProduced by Nick Hodson of London, England\n\n\n\n\nPercival Keene, by Captain Marryat.\n\n________________________________________________________________________\n\nCaptain Frederick Marryat was born July 10 1792, and died August 8 1848.\nHe retired from the British navy in 1828 in order to devote himself to\nwriting.  In the following 20 years he wrote 26 books, many of which are\namong the very best of English literature, and some of which are still\nin print.\n\nMarryat

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("JackFram/llama-68m", use_fast=True, legacy=False)

In [19]:
prompt = build_chat_input(tokenizer, "Please read a part of the book below, and then give me the summary!\n[start of the book]\n" + dataset['train'][0]['document']['text'][3:1024*356+473] + "\n[end of the book]\n\nNow you have read it! Please summarize it for me!")
prompt.shape

torch.Size([1, 102400])

In [20]:
tokenizer.decode(prompt[0])

'<s> Human: Please read a part of the book below, and then give me the summary!\n[start of the book]\nThe Project Gutenberg EBook of Percival Keene, by Frederick Marryat\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: Percival Keene\n\nAuthor: Frederick Marryat\n\nRelease Date: May 22, 2007 [EBook #21572]\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK PERCIVAL KEENE ***\n\n\n\n\nProduced by Nick Hodson of London, England\n\n\n\n\nPercival Keene, by Captain Marryat.\n\n________________________________________________________________________\n\nCaptain Frederick Marryat was born July 10 1792, and died August 8 1848.\nHe retired from the British navy in 1828 in order to devote himself to\nwriting.  In the following 20 years he wrote 26 books, many of wh